# MobileNet v4

In [30]:
torch.set_default_device('cuda')

In [1]:
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms

class CustomDataset(Dataset):
  def __init__(self, csv_file, transform=None):
    self.data = pd.read_csv(csv_file)
    self.transform = transform
    self.label_dict = {"notinfacted": 0, "infacted": 1}

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    # get image file paths from csv file
    img_name = self.data.iloc[idx, 0]
    # get the image from path
    image = Image.open(img_name)
    # get the image label from csv file
    label = self.data.iloc[idx, 1]
    # encode the image label accordingly
    encoded_label = self.label_dict[label]
    if self.transform:
      image = self.transform(image)

    return image, encoded_label


# Define transforms for the input images
transform = transforms.Compose(
    [
        transforms.Resize((244, 244)),
        transforms.ToTensor(),
        # normalization for color images
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


# Load train and test datasets
train_set = CustomDataset(
    csv_file="train.csv",
    transform=transform,
)
test_set = CustomDataset(
    csv_file="test.csv",
    transform=transform,
)

In [3]:
# Print set sizes
print(len(train_set))
print(len(test_set))

1030
257


In [28]:
from torch.utils.data import DataLoader
batch_size = 32

# Define data loaders
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=False, generator=torch.Generator(device='cuda'))
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, pin_memory=False, generator=torch.Generator(device='cuda'))

In [29]:
import torch
# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

# Define MobileNetV4 model for binary classification
class MobileNetV4(nn.Module):
    def __init__(self):
        super(MobileNetV4, self).__init__()
        self.model = models.mobilenet_v3_small(weights='MobileNet_V3_Small_Weights.DEFAULT')
        # Freeze all layers except the final classifier
        for param in self.model.parameters():
            param.requires_grad = False
        self.model.classifier[-1] = nn.Linear(self.model.classifier[-1].in_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.model(x)
        x = self.sigmoid(x)
        return x

# Initialize the model
model = MobileNetV4()

# Move the model to the appropriate device (GPU or CPU)
model = model.to(device)

In [32]:
# Define loss function and optimizer
loss_fn = nn.BCELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
import time

# Training loop
num_epochs = 20

start_time = time.time()
for epoch in range(num_epochs):
    epoch_start_time = time.time()

    # Set the model to training mode
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images).squeeze(dim=1)
        loss = loss_fn(outputs, labels.float())

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track the accuracy and loss
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct / total

    epoch_time_minutes = (time.time() - epoch_start_time) / 60.0
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Time: {epoch_time_minutes:.2f} minutes")
    # Save model checkpoints for each  epoch
    torch.save(model.state_dict(), f"mobileNetv4_model_bth{batch_size}_eph{num_epochs}_{epoch}.pth")

total_training_time_minutes = (time.time() - start_time) / 60.0
print(f"Total training time: {total_training_time_minutes:.2f} minutes")

Epoch [1/20], Loss: 0.5195, Accuracy: 0.7903, Time: 2.26 minutes
Epoch [2/20], Loss: 0.3131, Accuracy: 0.9155, Time: 2.26 minutes
Epoch [3/20], Loss: 0.2504, Accuracy: 0.9252, Time: 2.22 minutes
Epoch [4/20], Loss: 0.2098, Accuracy: 0.9417, Time: 2.18 minutes
Epoch [5/20], Loss: 0.1894, Accuracy: 0.9456, Time: 2.18 minutes
Epoch [6/20], Loss: 0.1789, Accuracy: 0.9515, Time: 2.22 minutes
Epoch [7/20], Loss: 0.1621, Accuracy: 0.9631, Time: 2.21 minutes
Epoch [8/20], Loss: 0.1537, Accuracy: 0.9602, Time: 2.21 minutes
Epoch [9/20], Loss: 0.1430, Accuracy: 0.9544, Time: 2.25 minutes
Epoch [10/20], Loss: 0.1446, Accuracy: 0.9602, Time: 2.75 minutes
Epoch [11/20], Loss: 0.1302, Accuracy: 0.9631, Time: 2.81 minutes
Epoch [12/20], Loss: 0.1639, Accuracy: 0.9621, Time: 2.71 minutes
Epoch [13/20], Loss: 0.1188, Accuracy: 0.9583, Time: 2.76 minutes
Epoch [14/20], Loss: 0.1048, Accuracy: 0.9767, Time: 2.69 minutes
Epoch [15/20], Loss: 0.1124, Accuracy: 0.9680, Time: 2.70 minutes
Epoch [16/20], Loss

# Evaluate Model

In [38]:
# Load the saved model state from the .pth file
saved_model_path = "mobileNetv4_model_bth32_eph20_13.pth"
model.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [39]:
# Set the model to evaluation mode
model.eval()

# Define a function to evaluate the model on the test loader
def evaluate_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze(dim=1)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# Evaluate the model on the test loader
test_accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9533


In [40]:
# Load the saved model state from the .pth file
saved_model_path = "mobileNetv4_model_bth32_eph20_19.pth"
model.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [41]:
# Set the model to evaluation mode
model.eval()

# Define a function to evaluate the model on the test loader
def evaluate_model(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).squeeze(dim=1)
            predicted = torch.round(outputs)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# Evaluate the model on the test loader
test_accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9572
